# Оптимизация стоимости виртуальных машин при миграции в облако

## Сценарий

Пришла пора смигрировать сервера из физизеского датацентра в облако? Один из шагов в этом нелегком деле - подбор размеров виртуальных машин в облаке. При этом, мы хотим уменьшить стоимость, но увеличить производительность наших виртуальных машин. Теоретически мы могли бы просто подобрать машины по размеру, по количеству CPU и RAM. И это, вроде, не самая сложная в мире задача. Однако в этом случае ни минимальная цена ни максимальная производительность не гарантированны. Хорошо, допустим мы всегда можем выбирать машины с минимальной ценой. Но в этом случае мы теряем производительность. Если же мы начнем выбирать машины с максимальной производительностью - вырастет и цена. Как с этим, быть? Возможно ли вообще получить максимальную производительность по минимальной цене? Давайте разбираться.

Но сначала выполним вот это, чтобы заработал [mermaid](https://mermaid-js.github.io/mermaid/#/./flowchart?id=flowcharts-basic-syntax) и [Microsoft Dataframe](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/). Они пригодится нам для простеньких диаграм и отображения табличных данных

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.21606.1

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Explain things visually using the Mermaid language . 
 This extension adds a new kernel that can render Mermaid markdown. This code will render a sequence diagram: 
 
 
#!mermaid
sequenceDiagram
 participant Alice
 participant Bob
 Alice->>John: Hello John, how are you?
 loop Healthcheck
 John->>John: Fight against hypochondria
 end
 Note right of John: Rational thoughts prevail!
 John-->>Alice: Great!
 John->>Bob: How about you?
 Bob-->>John: Jolly good!
 
 
 This extension also adds gestures to render a class diagram from any type. Use the ExploreWithUmlClassDiagram().Display(); extension method on System.Type to render its class diagram. 

 
 
typeof(List<string>).ExploreWithUmlClassDiagram().Display();

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Explore data visually using the nteract Data Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the nteract Data Explorer . Use the ExploreWithNteract() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithNteract().Display();
 
 

 To set the nteract Data Explorer use the following code 
 
 
 DataExplorer.SetDefault ();

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Explore data visually using the SandDance Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the SandDance Explorer . Use the ExploreWithSandDance() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithSandDance().Display();
 
 
 To set the SandDance Explorer use the following code 
 
 
 DataExplorer.SetDefault ();

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

## Minizinc

Как написано на [официальном сайте](https://www.minizinc.org/)

> MiniZinc is a free and open-source **constraint modeling language**.
>
> You can use MiniZinc to model constraint satisfaction and optimization problems in a **high-level**, **solver-independent** way, taking advantage of a large library of pre-defined constraints. Your model is then compiled into FlatZinc, a solver input language that is understood by a wide range of solvers.
>
> MiniZinc is developed at [Monash University](http://www.monash.edu/) in collaboration with [Data61 Decision Sciences](https://research.csiro.au/data61/tag/decision-sciences/) and the [University of Melbourne](http://unimelb.edu.au/).

Говоря простыми словами это, с одной стороны, специальный **декларативный** язык программирования, используемый для описания определенного класса оптимизационных задач. С другой стороны это, своего рода, абстракция над некоторым количеством `solvers` (решателей?- я буду использовать слово solver), что позволяет нам написать код один раз и попробовать его с разными solvers.

В нашем сценарии мы пытаемся решить оптимизационную задачу с ограничениями. И мы попытаемся использовать этот инструмент для ее решения.

## Что мы делаем

В этом разделе мы быстро проделаем наш эксперимент, посмотрим оценим и разберем его результат. Те же, кому будут интересны детали реализации смогут найти их в слежующем разделе. Чтобы все это работало на Windows системах, необходимо скачать и установить [minizinc](https://www.minizinc.org/). Кроме того, необходимо убедиться что путь к нему есть в **PATH**, поскольку функционал ниже пытается достать до minizinc именно так

> Для того чтобы спрятать излишнюю сложность, чась функционала, относительно несложная на самом деле, была спрятана [вот тут](./vm-optimization-minizinc/helperFunctions.ps1).

Начнем с простого описания. Наше решение состоит из [minizinc модели](./vm-optimization-minizinc/vmCostsCalculation-integer.mzn), написанной на соответвующем языке, [набора входных данных](./vm-optimization-minizinc/vmData.dzn) для этой модели и [скриптов](./vm-optimization-minizinc/helperFunctions.ps1), которые упрощаяют использование minizinc, передачу ему нужных данных и получение результатов. Входные данные содержат следующую информацию:

- данные об исходных серверах - их CPU, RAM и суммарные размеры дисков
- данные о виртуальных машинах в Azure из региона eastus2, включая . Про то как их выгрузить будет отдельный рассказ
- данные о дисках в Azure

Модель пытается **самостоятельно** подобрать подходящий размер виртуальной машины в облаке исходя из ограничений, описанных в модели для каждого исходного сервера. Например, модель требует чтобы объем RAM для соответствующей машины в облаке было больше или равно объему RAM исходной машины. Ключевое слово здесь - **самостоятельно**. Это означает что мы не говорим как подбирать, мы только задаем ограничение, требующее, чтобы это было так. Модель же, самостоятельно ищет решение, удовлетворяющее всем ограничениям. При этом, мы можем сказать модели, чтобы она пыталась либо минимизировать общую стоимость всех машин, либо максимизировать суммарную производительность, и учитывала это при выборе соотвествующих виртуальных машин.

При удачном стечении обстоятельств солвер гарантирует, что решение, которое он предложил, будет оптимальным, либо близким к оптимальному.

Чтобы использовать эту модель, загрузим наши функции - [dot sourcing](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_scripts?view=powershell-7.2#script-scope-and-dot-sourcing)

In [ ]:
. .\vm-optimization-minizinc\helperFunctions.ps1

Проведем три теста: 

- простой тест, где мы оптимизируем отдельно по цене и отдельно по производительности, чтобы иметь данные для сравнения
- сначала мнимизируем суммарную цену, а затем, зафиксировав ее, попытаемся максимизировать производительность. Это будет означать, что найдя минимум по цене, мы попытаемся подобрать такие машины, чтобы суммарная производительность увеличилась, не увеличивая цены
- сначала максимизируем производительность, и затем, с зафиксированной производительностью пытаться минимизировать ее стоимость. Это будет означать, что найдя максимум по производительности, мы попытаемся подобрать такие машины, чтобы суммарная цена уменьшилась, не уменьшая производительности

### Оптимизируем отдельно

Минимизируем по цене

In [ ]:
Start-MinizincVMOptimizationModel -Costs


totalPrice totalACU vmRecords
---------- -------- ---------
   42.4778     9200 {@{sourceVMName=vmN1; sourceVMCPU=2; sourceVMRAM=12; sourceVMDisk=460; selecte…



Максимизируем по производительности

In [ ]:
Start-MinizincVMOptimizationModel -Performance


totalPrice totalACU vmRecords
---------- -------- ---------
   146.496    18860 {@{sourceVMName=vmN1; sourceVMCPU=2; sourceVMRAM=12; sourceVMDisk=460; selecte…



Как видно из этих двух тестов, в первом случае мы получили некую суммарную цену, которая значительно меньше суммарной цены во втором случае. При этом видно, что увеличение производительность в два раза, дает увеличение цены почти более чем в 3 раза. Возникает вопрос, а можно ли лучше?

### Минимизируем цену в ущерб производительности

В качестве оценки производительнсть виртуальной машины будем использовать некий синтетический параметр, который Microsoft называет [Azure compute unit (ACU)](https://docs.microsoft.com/en-us/azure/virtual-machines/acu). В наших экспериментах мы будем пытаться его максимизировать, стараясь сохранить минимальную цену.

В этом эксперименте мы  сначала минимизируем общую цену, и затем, зафиксировав это значение, максимизируем ACU

In [ ]:
$ret = Start-MinizincVMOptimizationModel -Costs | Start-MinizincVMOptimizationModel -Performance
$ret


totalPrice totalACU vmRecords
---------- -------- ---------
   42.4778     9410 {@{sourceVMName=vmN1; sourceVMCPU=2; sourceVMRAM=12; sourceVMDisk=460; selecte…



Здесь мы видим, что минимальная цена осталась минимальной, но модель смогла найти выриант, при котором суммарная производительность стала немного больше

Давайте посмотрим, что же подобрала нам модель. Для этого воспользуемся [DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/). Прото потому что .Net Interactive умеет его красиво отображать. Для этого просто выбросим наши результаты в csv строку, создадим соответсвующий объект и дернем extension method. Синтаксис кривоват, но что поделать, PowerShell не очень любит extension methods

In [ ]:
$csv = ($ret.vmRecords | ConvertTo-Csv -NoTypeInformation) -join "`n"
$df = [Microsoft.Data.Analysis.DataFrame]::LoadCsvFromString($csv)
[Microsoft.DotNet.Interactive.Kernel]::display($df)

index,sourceVMName,sourceVMCPU,sourceVMRAM,sourceVMDisk,selectedSize,targetVMCPU,targetVMRAM,targetVMDisk,targetVMPrice,targetVMACU
0,vmN1,2,12,460,Standard_A2m_v2,2,16,4,0.18,100
1,vmN2,2,24,1440,Standard_A4m_v2,4,32,4,0.378,100
2,vmN3,2,4,110,Standard_A2_v2,2,4,4,0.136,100
3,vmN4,6,23,260,Standard_D8s_v4,8,32,4,0.752,195
4,vmN5,3,16,1120,Standard_D4s_v4,4,16,4,0.376,195
5,vmN6,2,32,200,Standard_A4m_v2,4,32,4,0.378,100
6,vmN7,2,7,267,Standard_A2m_v2,2,16,4,0.18,100
7,vmN8,3,32,134,Standard_A4m_v2,4,32,4,0.378,100
8,vmN9,2,8,213,Standard_A2m_v2,2,16,4,0.18,100
9,vmN11,2,4,77,Standard_A2_v2,2,4,4,0.136,100



MimeTypes
---------
{text/html}



Тут стоит обратить внимание на то, что `targetVMACU` везде 100. Это, наверное, и послужило прибавкой в суммарной производительности

К слову сказать, при отображении можно обойтись и без DataFrame. Как показано ниже, но для больших таблиц это может быть не очень удобно.

In [ ]:
$view = $ret.vmRecords | ConvertTo-Html -Fragment
[Microsoft.DotNet.Interactive.Kernel]::HTML($view) | Out-Display

### Минимизируем цену, без потери производительности

Теперь попробуем другой подход. Сначала максимизируем производительность, зафиксируем ее и попытаемся минимизировать цену не теряя производительности

In [ ]:
$ret2 = Start-MinizincVMOptimizationModel -Performance | Start-MinizincVMOptimizationModel -Costs
$ret2


totalPrice totalACU vmRecords
---------- -------- ---------
    54.936    18860 {@{sourceVMName=vmN1; sourceVMCPU=2; sourceVMRAM=12; sourceVMDisk=460; selecte…



И, о ужас, в наше конкретном случае оказывается, что увеличив цену всего на ~20% мы можем получить увеличение суммарной производительности в два раза! Это успех я щетаю! Как же так вышло?

In [ ]:
$csv = ($ret2.vmRecords | ConvertTo-Csv -NoTypeInformation) -join "`n"
$df2 = [Microsoft.Data.Analysis.DataFrame]::LoadCsvFromString($csv)
[Microsoft.DotNet.Interactive.Kernel]::display($df2)

index,sourceVMName,sourceVMCPU,sourceVMRAM,sourceVMDisk,selectedSize,targetVMCPU,targetVMRAM,targetVMDisk,targetVMPrice,targetVMACU
0,vmN1,2,12,460,Standard_E4_2as_v4,4,32,4,0.436,230
1,vmN2,2,24,1440,Standard_E4_2as_v4,4,32,4,0.436,230
2,vmN3,2,4,110,Standard_E4_2as_v4,4,32,4,0.436,230
3,vmN4,6,23,260,Standard_E8_2as_v4,8,64,4,0.872,230
4,vmN5,3,16,1120,Standard_E4_2as_v4,4,32,4,0.436,230
5,vmN6,2,32,200,Standard_E4_2as_v4,4,32,4,0.436,230
6,vmN7,2,7,267,Standard_E4_2as_v4,4,32,4,0.436,230
7,vmN8,3,32,134,Standard_E4_2as_v4,4,32,4,0.436,230
8,vmN9,2,8,213,Standard_E4_2as_v4,4,32,4,0.436,230
9,vmN11,2,4,77,Standard_E4_2as_v4,4,32,4,0.436,230



MimeTypes
---------
{text/html}



Модель выбрала совсем другие размеры виртуальных машин, но при этом старалась держать цену минимально возможной

## Как это на самом деле работает

Этот раздел для тех кто хочет немного погрузиться в детали. Однако деталей так много, что лучше погружаться в них вот тут:

- [Basic Modeling for Discrete Optimization](https://www.coursera.org/learn/basic-modeling)
- [Advanced Modeling for Discrete Optimization](https://www.coursera.org/learn/advanced-modeling)


Ниже представлена mermaid диаграмма всей штуки.

In [ ]:
#!mermaid
flowchart LR
    data[minizinc data] --> pwsh[PowerShell Wrapper]
    model[minizinc model] --> pwsh[PowerShell Wrapper]
    pwsh -.calls.-> mz[minizinc.exe]
    mz --uses--> solver[solver GECODE]
    mz -.returns results.-> pwsh
    pwsh --parses and returns-->r[results]

    source["source servers details (CPU, RAM, Disks)"] -.-> data
    azureVM["Azure VM pricing and ACU"]  -.-> data
    azureDisk["Azure Disk prices"]  -.-> data

Основная часть "Марлезонского балета" - [модель](./vm-optimization-minizinc/vmCostsCalculation-integer.mzn). Как уже упоминалось, модель декларативно описывает результат. И в этом ее прелесть. Нам не хочется писать, как она должна искать решение. Мы описывем модель в виде набора ограничений и целевой функции. Давайте посмотрим на примерах, опустив описание переменных и типов даных

Первое ограничение говорит о том, что для всех размеров виртуальных машин, которые подберет модель, требуется, чтобы объем RAM выбранной машины был больше или равен RAM исходной машины

```console
constraint forall(vm in existingVMs)(
    vmSizeRAM[selectedSize[vm]] >= vmRAM[vm]
);
```

То, же самое и для CPU, но с допущением, что по CPU, а это самый дорогой ресурс, мы разрешаем "просесть" на 20%

```console
constraint forall(vm in existingVMs)(
   vmSizeCPU[selectedSize[vm]] >=  vmCPU[vm] * 0.8
);
```

Кроме этого у нас есть два целевых функции, которые мы собираемся оптимизировать. Каждая из них просто подсчитывает соотвествующую сумму: цена всех выбранных машин и производительность.

```console
var int: totalPrice = sum(vm in existingVMs)(vmSizePrice[selectedSize[vm]]);
var int: totalACU = sum(vm in existingVMs)( vmSizeACU[selectedSize[vm]] );
```

И, наконец, само важное и интересное. То, что называется decision variables, попросту - решения. Строка ниже говорит, что в качестве решения мы хотим получить массив, в котором для каждой их исходных машин будет находиться выбранный моделью размер соответствующей машины в облаке.

```console
array[existingVMs] of var vmSizes: selectedSize;
```

Чтобы все это заработало в модели не хватает указания, что именно мы оптимизируем. Для этого в ней обязательно должен присутствовать оператор `solve`. Есть несколько вариантов поиска решений: 

- `solve satisfy` - ищет любое/или все решения, удовлетворяющее ограничениям. 
- `solve minimize ...` - пытается минимизировать некую целевую функцию, например суммарную цену
- `solve maximize ...` - пытается максимизировать некую целевую функцию, например суммарную производительность

В нашем случае команда `Start-MinizincVMOptimizationModel` просто вставляет в модель подходящий оператор `solve` и запускает minizinc.

Для того чтобы оптимизировать по нескольком параметрам нам нужно сначала оптимизировать по дному из них, зафиксировать это в виде ограничения и, затем, оптимизировать по второму. То есть команды `Start-MinizincVMOptimizationModel -Costs | Start-MinizincVMOptimizationModel -Performance` делают вот что:

1. в модель вставляется операцию `solve  minimize totalPrice;`
2. модель исполняется, извлекается результат из minizinc, парсится его и возвращается в PowerShell pipeline
3. результат по pipeline передается во второй вызов который делает из него новое ограничение
4. в новую модель добавляется ограничение, содержащее результат предыдущей оптимизации и запрос на новую - `constraint totalACU >= $($InputObject.totalACU); solve  minimize totalPrice;` или `constraint totalPrice <= $($InputObject.totalPrice * 10000); solve  maximize totalACU;` где `$($InputObject.xxx)` это результа предыдущей операции
5. модель исполняется, извлекается результат из minizinc, парсится его и возвращается в PowerShell pipeline

В конечном итоге происходит примерно следующее. На входе у модели есть размеры исходных машин, и все возможные размеры соответствующих машин в облаке. Для каждой из исходных, модель пытается подобрать такую машину в облаке, которая удовлетворяет заданным ограничениям по CPU и RAM. При этом рассчитываются суммарные цена и производительность и из всех возможных комбинаций выбираются те, для которых суммы удовлетворяют нашим требованиям максимальности и минимальности.